In [ ]:
! pip install apache-beam

In [1]:
import apache_beam as beam
import logging
import requests
from urllib.error import HTTPError


In [2]:


class callAPI(beam.DoFn):
    def process(self, element):
        fro = element.get("from")
        to = element.get("to")

        url = 'http://127.0.0.1:8000/content?from={}&to={}'.format(fro, to)
        res = requests.get(url)
        data  = res.json()
        yield data


In [10]:
with beam.Pipeline() as p:
    data = p | beam.Create(values= [{'from': 1, 'to': 20}])
    ap_response = data | beam.ParDo(callAPI())
    #ap_response | "Print results" >> beam.Map(print)
    ap_response | "Print results" >> beam.io.WriteToText('data/twenty.txt')

In [14]:
import csv
from io import StringIO

# Read the data from the file
with open('data/twenty.txt-00000-of-00001', 'r') as file:
    data = eval(file.read())  # Assuming the file contains valid Python data

csv_output = StringIO()

# Define the CSV header based on keys of the dictionaries in the nested list
csv_header = data[0][0].keys()

# Initialize the CSV writer
csv_writer = csv.DictWriter(csv_output, fieldnames=csv_header)

# Write the header
csv_writer.writeheader()

# Write each dictionary as a row
for nested_list in data:
    for dictionary in nested_list:
        csv_writer.writerow(dictionary)

# Get the CSV content as a string
csv_content = csv_output.getvalue()

# Write the CSV content to a file
with open('output.csv', 'w') as csv_file:
    csv_file.write(csv_content)

    
print("CSV file 'output.csv' has been created.")

CSV file 'output.csv' has been created.
